In [335]:
import pandas as pd

# Open the csv file
df = pd.read_csv('Annotations.csv')

# Remove SensorSerial, SensorLocation, SensorValue, SensorType
df.head()

,DateTime,Location,Activity,scaledTime,Weekday,Weekend
0,2011-06-15 00:09:20.394217,Bedroom,Other_Activity,12:09:20,Wednesday,0
1,2011-06-15 00:09:21.556337,Bedroom,Other_Activity,12:09:21,Wednesday,0
2,2011-06-15 00:09:22.064266,Bedroom,Other_Activity,12:09:22,Wednesday,0
3,2011-06-15 00:09:24.750548,Bedroom,Other_Activity,12:09:24,Wednesday,0
4,2011-06-15 00:09:33.499231,Ignore,Other_Activity,12:09:33,Wednesday,0


In [336]:

# Duration from start to end in minutes
start = df['DateTime'][0]
end = df['DateTime'][len(df)-1]
duration = (pd.to_datetime(end) - pd.to_datetime(start)).total_seconds()/60
print(duration)


import time
data = {
    'Activity': [],
    'Start time': [],
    'Duration': [], # in minutes
    'Number of interruptions': [], # Number of times sleep was interrupted. Over 60 minutes, considered a new activity
    'Duration of interruptions': [], # in minutes
    'Day of activity': [], # 0 = Monday --> 6 = Sunday
    'Weekend or weekday': [], # 0 = weekday, 1 = weekend
}

# Initialize variables
activity = df['Activity'][0]
start_time = df['DateTime'][0].split('.')[0]
num_interruptions = 0
duration_of_interruptions = 0
weekday = df['Weekday'][0]
weekend = df['Weekend'][0]

exact_time = time.strptime(start_time, '%Y-%m-%d %H:%M:%S')
print(exact_time)


# Iterate through the rows. Cannot save duration of activity until the next activity is found
for i in range(len(df)):
    if activity != df['Activity'][i]:
        # New activity found, save the previous activity
        data['Activity'].append(activity)
        data['Start time'].append(start_time)
        data['Duration'].append((time.mktime(time.strptime(df['DateTime'][i].split('.')[0], '%Y-%m-%d %H:%M:%S')) - time.mktime(exact_time)) / 60)
        data['Number of interruptions'].append(num_interruptions)
        data['Duration of interruptions'].append(duration_of_interruptions)
        data['Day of activity'].append(weekday)
        data['Weekend or weekday'].append(weekend)

        # Reset variables
        activity = df['Activity'][i]
        start_time = df['DateTime'][i].split('.')[0]
        num_interruptions = 0
        duration_of_interruptions = 0
        weekday = df['Weekday'][i]
        weekend = df['Weekend'][i]
        exact_time = time.strptime(start_time, '%Y-%m-%d %H:%M:%S')
        
df_activity = pd.DataFrame(data)        
df_activity.head()
print('Sum of durations: ', df_activity['Duration'].sum())
print('Size of df_activity: ', len(df_activity))

# If duration of activity is less than 1 minute, remove it
df_activity = df_activity[df_activity['Duration'] > 1]
print('Sum of durations: ', df_activity['Duration'].sum())
print('Size of df_activity: ', len(df_activity))

87804.90170181666
time.struct_time(tm_year=2011, tm_mon=6, tm_mday=15, tm_hour=0, tm_min=9, tm_sec=20, tm_wday=2, tm_yday=166, tm_isdst=-1)
Sum of durations:  87804.90000000001
Size of df_activity:  15255
Sum of durations:  86644.43333333333
Size of df_activity:  2747


In [337]:
# Check if activity is sleep 60 minutes after the last sleep activity
def checkSleep(df):
    df_new = pd.DataFrame(columns=df.columns)
    i = 0
    while i < len(df) - 1:
        # Calculate progress
        print('{}/{}'.format(i, len(df) - 1), end='\r')

        current_activity = df.iloc[i]['Activity']
        next_activity = df.iloc[i+1]['Activity']

        if current_activity == 'Sleep':
            j = i + 1

            if j > len(df) - 1:
                print('Break')
                break
            
            interuption_duration = df.iloc[j]['Duration']
            while interuption_duration < 60:
                j += 1
                interuption_duration += df.iloc[j]['Duration']
            interuption_duration -= df.iloc[j]['Duration']

            if df.iloc[j]['Activity'] == 'Sleep':
                # Interrupted sleep activity
                num_interruptions = j - i - 1

                # Merge the sleep activities
                new_row = {
                    'Activity': 'Sleep',
                    'Start time': df.iloc[i]['Start time'],
                    'Duration': df.iloc[i]['Duration'] + df.iloc[j]['Duration'],
                    'Number of interruptions': df.iloc[i]['Number of interruptions'] + df.iloc[j]['Number of interruptions'] + num_interruptions,
                    'Duration of interruptions': df.iloc[i]['Duration of interruptions'] + df.iloc[j]['Duration of interruptions'] + interuption_duration,
                    'Day of activity': df.iloc[i]['Day of activity'],
                    'Weekend or weekday': df.iloc[i]['Weekend or weekday']
                }
                df_new = df_new.append(new_row, ignore_index=True)
                i = j + 1
            
            else:
                # New activity
                df_new = df_new.append(df.iloc[i], ignore_index=True)
                i += 1

        else:
            # Not a sleep activity
            df_new = df_new.append(df.iloc[i], ignore_index=True)
            i += 1

    # Add the last activity
    df_new = df_new.append(df.iloc[len(df)-1], ignore_index=True)
    return df_new

total_interruptions = df_activity['Number of interruptions'].sum()
new_total_interruptions = 10

while total_interruptions != new_total_interruptions:
    total_interruptions = new_total_interruptions
    df_activity = checkSleep(df_activity)
    new_total_interruptions = df_activity['Number of interruptions'].sum()

    print('Sum of durations: ', df_activity['Duration'].sum())
    print('Sum of interruptions: ', df_activity['Number of interruptions'].sum())
    print('Sum of duration of interruptions: ', df_activity['Duration of interruptions'].sum())
    print('Sum of total duration: ', df_activity['Duration'].sum() + df_activity['Duration of interruptions'].sum())
    print('Size of df_activity: ', len(df_activity))


Sum of durations:  86423.66666666667
Sum of interruptions:  59
Sum of duration of interruptions:  220.7666666666667
Sum of total duration:  86644.43333333333
Size of df_activity:  2648
Sum of durations:  86390.54999999999
Sum of interruptions:  68
Sum of duration of interruptions:  253.88333333333333
Sum of total duration:  86644.43333333332
Size of df_activity:  2632
Sum of durations:  86390.54999999999
Sum of interruptions:  68
Sum of duration of interruptions:  253.88333333333333
Sum of total duration:  86644.43333333332
Size of df_activity:  2632


In [338]:
# Change time to HH:MM:SS
def changeTime(time):
    return time.split(' ')[1]

df_activity['Start time short'] = df_activity['Start time'].apply(changeTime)

In [339]:
# Save the dataframe to a csv file
df_activity.to_csv('Activity.csv', index=False)

In [340]:
import plotly.graph_objects as go
import plotly.express as px

# Plot the data for one day
df_day = df_activity[df_activity['Start time'].str.contains('2011-06-20')]

# Add row to beginning of dataframe for sleep 00:00:00 to start time
new_row = {
    'Activity': 'Sleep',
    'Start time': '2011-06-20 00:00:00',
    'Duration': (time.mktime(time.strptime(df_day.iloc[0]['Start time'], '%Y-%m-%d %H:%M:%S')) - time.mktime(time.strptime('2011-06-20 00:00:00', '%Y-%m-%d %H:%M:%S'))) / 60,
    'Number of interruptions': 0,
    'Duration of interruptions': 0,
    'Day of activity': df_day.iloc[0]['Day of activity'],
    'Weekend or weekday': df_day.iloc[0]['Weekend or weekday'],
    'Start time short': '00:00:00'
}

df_day = df_day.append(new_row, ignore_index=True)
df_day = df_day.sort_values(by=['Start time'])

# Set end time of each activity as start time of next activity
df_day['End time'] = df_day['Start time'].shift(-1)
df_day['End time'] = df_day['End time'].fillna('2011-06-20 23:59:59')

# df_day_inversed = clean_df(df_day)
# df_day_inversed = df_day_inversed.sort_values(by=['Start time'], ascending=False)
# df_day_inversed['End time'] = df_day_inversed['Start time'].shift(-1)

data = []
for i in range(len(df_day)):
    if i < len(px.colors.qualitative.Pastel):
        j = i
    else:
        j = i % len(px.colors.qualitative.Pastel)
    data.append(dict(
        Activity=df_day.iloc[i]['Activity'], 
        Start=df_day.iloc[i]['Start time'],
        Finish=df_day.iloc[i]['End time'],
    ))

df = pd.DataFrame(data)
# Set color palette
fig = px.timeline(df, x_start="Start", x_end="Finish", y="Activity", color="Activity", color_discrete_sequence=px.colors.qualitative.Pastel)

# Leave date out of x-axis, only show time
fig.update_xaxes(
    tickformat='%H:%M:%S',
    title_text='Time',
    title_font_size=20,
    tickfont_size=20,
)

fig.update_yaxes(
    title_text='Activity',
    title_font_size=20,
    tickfont_size=20,
)

fig.update_layout(showlegend=False)
fig.show()

fig.write_image("Activity.png", width=1500, height=800, scale=3)



In [392]:
# Import standardisation libraries
from sklearn.preprocessing import StandardScaler

df_activity = pd.read_csv('Activity.csv')
df_activity_clean = df_activity.copy()
def clean_df(df_activity_clean):

    df_activity_clean['Start time short']
    # If hour > 12, 24 - hour
    def changeTime(time):
        try:
            hour = int(time.split(':')[0])
            minute = int(time.split(':')[1])
            second = int(time.split(':')[2])
            if hour > 11:
                hour = hour - 24
                minute = 60 - minute
                second = 60 - second
        except Exception as e:
            print('time: ', time)
        return hour + minute/60 + second/3600

    df_activity_clean['Start time'] = df_activity_clean['Start time short'].apply(changeTime)
    df_activity_clean.drop(columns=['Start time short'], inplace=True)

    return df_activity_clean

# df_activity = clean_df(df_activity_clean)
# df_activity.drop(columns=['Start time short'], inplace=True)

# Training data is first 31 days
day1 = '2011-06-15'
day31 = '2011-07-15'

# Convert time to float
def convertTime(time):
    time = time.split(' ')[1]
    hour = int(time.split(':')[0])
    minute = int(time.split(':')[1])
    second = int(time.split(':')[2])
    return hour + minute / 60 + second / 3600

def isDayBefore(start_time):
    timeConverted = time.strptime(start_time, '%Y-%m-%d %H:%M:%S')
    return timeConverted < time.strptime(day31, '%Y-%m-%d')

# Test data is last 31 days
def isDayAfter(start_time):
    timeConverted = time.strptime(start_time, '%Y-%m-%d %H:%M:%S')
    return timeConverted > time.strptime(day31, '%Y-%m-%d')

# All activities split into training and test data
df_train = df_activity[df_activity['Start time'].apply(isDayBefore)]
df_test = df_activity[df_activity['Start time'].apply(isDayAfter)]

# Reset index
df_test.reset_index(drop=True, inplace=True)

# Save df
df_train.to_csv('Unscaled/df_train.csv', index=False)
df_test.to_csv('Unscaled/df_test.csv', index=False)

# Sleep only
df_train_sleep = df_train[df_train['Activity'] == 'Sleep']
df_train_sleep = df_train_sleep.drop(columns=['Activity'])
df_test_sleep = df_test[df_test['Activity'] == 'Sleep']
df_test_sleep = df_test_sleep.drop(columns=['Activity'])

# Save df
df_train_sleep.to_csv('Unscaled/df_train_sleep.csv', index=False)
df_test_sleep.to_csv('Unscaled/df_test_sleep.csv', index=False)

df_train['Start time'] = df_train['Start time'].apply(convertTime)
df_test['Start time'] = df_test['Start time'].apply(convertTime)

# Transform day of week to number
def dayOfWeek(day):
    if day == 'Monday':
        return 0
    elif day == 'Tuesday':
        return 1
    elif day == 'Wednesday':
        return 2
    elif day == 'Thursday':
        return 3
    elif day == 'Friday':
        return 4
    elif day == 'Saturday':
        return 5
    elif day == 'Sunday':
        return 6

df_train['Day of activity'] = df_train['Day of activity'].apply(dayOfWeek)
df_test['Day of activity'] = df_test['Day of activity'].apply(dayOfWeek)
df_train_sleep['Day of activity'] = df_train_sleep['Day of activity'].apply(dayOfWeek)
df_test_sleep['Day of activity'] = df_test_sleep['Day of activity'].apply(dayOfWeek)



In [343]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder


# Clean data
df_train_sleep = clean_df(df_train_sleep)
df_test_sleep = clean_df(df_test_sleep)

# Create string of comma separated column names
header = ','.join(df_train_sleep.columns)

# Standardise the data
scaler = StandardScaler()
scaler.fit(df_train_sleep)
df_train_sleep = scaler.transform(df_train_sleep)
df_test_sleep = scaler.transform(df_test_sleep)

# Save arrays to csv files with headers
np.savetxt('df_train_sleep.csv', df_train_sleep, delimiter=',', header=header, comments='')
np.savetxt('df_test_sleep.csv', df_test_sleep, delimiter=',', header=header, comments='')


In [393]:
print(df_test)


                Activity  Start time    Duration  Number of interruptions  \
0                 Toilet    6.453056    3.633333                        0   
1         Other_Activity    6.522500    1.383333                        0   
2         Other_Activity    6.550000    4.850000                        0   
3                  Bathe    6.630833    7.816667                        0   
4                  Groom    6.761111    4.683333                        0   
...                  ...         ...         ...                      ...   
1150  Wash_Dinner_Dishes   20.755833    1.216667                        0   
1151    Work_On_Computer   20.778611   53.966667                        0   
1152    Personal_Hygiene   21.684722    2.233333                        0   
1153               Dress   21.743611    2.150000                        0   
1154               Sleep   21.781389  106.983333                        0   

      Duration of interruptions  Day of activity  Weekend or weekday  \
0  

In [396]:


unique_activities = df_train['Activity'].unique()
activity_dict = [{'Activity': activity, 'Number': i} for i, activity in enumerate(unique_activities)]

# Replace activity with number in training and test data
def replaceActivity(activity):
    for i in range(len(activity_dict)):
        if activity == activity_dict[i]['Activity']:
            return activity_dict[i]['Number']


# One hot encode activity
df_train_onehot = df_train.copy()
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(df_train_onehot[['Activity']])
df_train_onehot = enc.transform(df_train_onehot[['Activity']]).toarray()
df_train_onehot = pd.DataFrame(df_train_onehot, columns=['Activity_' + str(i) for i in range(len(unique_activities))])
columns = df_train.columns
columns = columns.drop('Activity')
df_train_onehot = pd.concat([df_train_onehot, df_train[columns]], axis=1)


df_test_onehot = df_test.copy()
df_test_onehot = enc.transform(df_test_onehot[['Activity']]).toarray()
df_test_onehot = pd.DataFrame(df_test_onehot, columns=['Activity_' + str(i) for i in range(len(unique_activities))])
columns = df_test.columns
columns = columns.drop('Activity')
df_test_onehot = pd.concat([df_test_onehot, df_test[columns]], axis=1)

# Clean data
df_train_onehot = clean_df(df_train_onehot)
df_test_onehot = clean_df(df_test_onehot)

# Create string of comma separated column names
header = ','.join(df_train_onehot.columns)

# Standardise the data
scaler = StandardScaler()
scaler.fit(df_train_onehot)
df_train_onehot = scaler.transform(df_train_onehot)
df_test_onehot = scaler.transform(df_test_onehot)

# Save arrays to csv files
np.savetxt('df_train_onehot.csv', df_train_onehot, delimiter=',', header=header, comments='')
np.savetxt('df_test_onehot.csv', df_test_onehot, delimiter=',', header=header, comments='')


In [ ]:

# Replace activity with number
df_train['Activity'] = df_train['Activity'].apply(replaceActivity)
df_test['Activity'] = df_test['Activity'].apply(replaceActivity)

# Clean data
df_train = clean_df(df_train)
df_test = clean_df(df_test)

# Create string of comma separated column names
header = ','.join(df_train.columns)

# Standardise the data
scaler = StandardScaler()
scaler.fit(df_train)
df_train = scaler.transform(df_train)
df_test = scaler.transform(df_test)

# Save arrays to csv files
np.savetxt('df_train.csv', df_train, delimiter=',', header=header, comments='')
np.savetxt('df_test.csv', df_test, delimiter=',', header=header, comments='')